# Grover Amplitude Amplification for Quantum Integration

This notebook implements:
1. State preparation using Isometry (Knill decomposition)
2. Range oracle using IntegerComparator for arbitrary intervals [a, b]
3. Grover diffusion operator for amplitude amplification

**Author:** Miro (teammate)

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import IntegerComparator
import numpy as np
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit.circuit.library import Isometry

In [ ]:
def state_prep(n, f_x_func):          # Uses Knill decomposition. Scales like 2^n/n gates on n qubits
    """
    Evaluates f_x on a periodic grid and prepares the quantum state.
    """
    N = 2**n
    x = np.linspace(0, 1, N, endpoint=False)
    
    # 1. Evaluate the function on the main grid AND periodic neighbors
    # This ensures periodic boundary conditions
    target = (f_x_func(x) + 
              f_x_func(x + 1) + 
              f_x_func(x - 1))
    
    # 2. Now calculate the norm of the resulting VECTOR
    norm_val = np.linalg.norm(target)
    if norm_val > 0:
        target = target / norm_val

    # 3. Use Isometry for perfect amplitude mapping
    iso = Isometry(target, 0, 0)
    qc = QuantumCircuit(n)
    qc.append(iso, range(n))
    IsomeryQC=qc
    
    return IsomeryQC, target, x

# --- Test and Plot ---
n = 5
a=0.2
b=0.75
u0 = lambda x: np.sin(2*np.pi*x)**2 #np.exp(-50 * (x - 1/3)**2)

# Call the function
IsomeryQC, target_vec, x_vals = state_prep(n, u0)

# Simulate state
state = Statevector.from_instruction(IsomeryQC)
q_amps = np.abs(state.data)

# --- Plotting ---
plt.figure(figsize=(10, 5))
plt.plot(x_vals, target_vec, 'r--', label='Ideal Periodic Target', alpha=0.6)
plt.step(x_vals, q_amps, where='mid', label='Quantum Amplitudes', color='blue')
plt.title(f"Successful State Prep: {n} Qubits")
plt.xlabel("Position x")
plt.ylabel("Amplitude u")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"L2 Error: {np.linalg.norm(target_vec - q_amps):.2e}")
print(IsomeryQC.draw(output='text'))

In [ ]:
def range_oracle(n, c, d):

    a, b = np.ceil(c*(2**n)), np.floor(d*(2**n))
    
    # Determine how many qubits the comparator actually needs
    # IntegerComparator needs: n (input) + 1 (result) + (n-1) (internal ancillas)
    temp_comp = IntegerComparator(n, a)
    num_comp_qubits = temp_comp.num_qubits
    
    x = QuantumRegister(n, 'x')
    # We need enough ancillas for TWO comparisons simultaneously if we want to CCX them,
    # OR we can reuse the same ancillas by being careful. 
    # Let's use two distinct result qubits and one shared pool of internal ancillas.
    res_a = QuantumRegister(1, 'res_a')
    res_b = QuantumRegister(1, 'res_b')
    target = QuantumRegister(1, 'target')
    
    # Internal ancillas needed by the IntegerComparator (usually n-1)
    internal_ancillas = QuantumRegister(num_comp_qubits - n - 1, 'internal')
    
    qc = QuantumCircuit(x, res_a, res_b, internal_ancillas, target)
    
    # Define the comparison logic
    # x >= a
    comp_a = IntegerComparator(n, a, geq=True)
    # x <= b (which is x < b+1)
    comp_b = IntegerComparator(n, b + 1, geq=False)
    
    # Step 1: Compute x >= a
    # Mapping: x-qubits + result-qubit + internal-ancillas
    qc.append(comp_a, x[:] + res_a[:] + internal_ancillas[:])
    
    # Step 2: Compute x <= b
    qc.append(comp_b, x[:] + res_b[:] + internal_ancillas[:])
    
    # Step 3: Target = res_a AND res_b
    qc.x(target)
    qc.h(target)
    qc.ccx(res_a, res_b, target)
    
    # Step 4: Uncompute to clean up res and internal ancillas
    qc.append(comp_b.inverse(), x[:] + res_b[:] + internal_ancillas[:])
    qc.append(comp_a.inverse(), x[:] + res_a[:] + internal_ancillas[:])
    
    return qc

def first_n_indices(n):
    return list(range(n))

# 1. Generate the list of target indices [0, 1, ..., n-1]
target_qubits = first_n_indices(n)

my_oracle = range_oracle(n, a, b)

# 2. Add the small circuit onto the big one
# We use inplace=True to modify my_oracle directly
my_oracle.compose(IsomeryQC, qubits=target_qubits, front=True, inplace=True)

print(my_oracle.draw(output='text'))

In [ ]:
state = Statevector(my_oracle)
amplitudes = state.data

# 4. Print Results
print(f"{'Index':<6} | {'Binary':<8} | {'Amplitude (Real + Imag)':<25} | {'Phase':<8} | {'Prob':<8}")
print("-" * 85)

for i in range(2**n):
    amp = amplitudes[i]
    prob = np.abs(amp)**2
    phase = np.angle(amp, deg=True)
    bin_str = format(i, f'0{n}b')
    
    marker = " <--- IN RANGE" if np.ceil(a*2**n) <= i <= np.floor(b*2**n) else ""
    
    # Color coding for terminal output
    row_color = "\033[92m" if marker else "" 
    reset = "\033[0m"
    
    print(f"{row_color}{i:<6} | {bin_str:<8} | {amp.real:>7.4f} + {amp.imag:>7.4f}j | {phase:>6.1f}° | {prob:>6.4f} {marker}{reset}")

In [ ]:
def add_diffusion(qc, n):
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1)
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))

add_diffusion(my_oracle, n)

print(my_oracle.draw(output='text'))